# Retrival 실험

실험 주제 : Train, Valid Dataset에 있는 Question, Context를 가져와서 Sparse, Dense, ElasticSearch를 활용해서 성능을 비교해 보려고한다.

1. TF-IDF
2. rank25 bm25 module
    - bm25okapi
    - bm25+
    - bm25L
    - bm25-Adapt
    - bm25T
3. ElasticSearch
    - Nori_tokenizer 사용했다.

# Dataset 불러오기

In [1]:
import os
import json
import pandas as pd

wikidocs_path = '../../data/wikipedia_documents.json'
train_file_path = '../../train.csv'
valid_file_path = '../../valid.csv'

train_data = pd.read_csv(train_file_path)
valid_data = pd.read_csv(valid_file_path)

with open('../../data/wikipedia_documents.json') as data_file:
    local = json.load(data_file)

wiki_corpus = []

for i in range(len(local)):
    wiki_corpus.append(local[str(i)])

    
data = pd.concat([train_data, valid_data])
wiki_data = pd.DataFrame(wiki_corpus)

In [2]:
data

,answer_start,answer_text,context,document_id,id,question,title
0,235,하원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,18293,mrc-1-000067,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,미국 상원
1,212,《경영의 실제》,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,51638,mrc-0-004397,현대적 인사조직관리의 시발점이 된 책은?,인사조직관리
2,510,백성,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,5028,mrc-1-000362,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,강희제
3,625,중국,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",34146,mrc-0-001510,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,금동삼존불감
4,30,4개,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,47334,mrc-0-000823,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,계사명 사리구
...,...,...,...,...,...,...,...
235,1084,제 양왕,"연나라 군대의 사령관이 악의에서 기겁으로 교체되자, 전단은 스스로 신령의 계시를 받...",53265,mrc-0-000484,전단이 연나라와의 전쟁에서 승리했을 당시 제나라의 왕은 누구인가?,전단 (제나라)
236,343,'일곱 개의 신전 광장',현재 우리가 볼 수 있는 티칼의 모습은 펜실베이니아 대학교와 과테말라 정부의 협조 ...,12190,mrc-0-002095,공놀이 경기장 중 일부는 어디에 위치하고 있나?,티칼
237,247,미나미 지로,1940년 5월 1일 오전 창씨개명에 비협조적이라는 이유로 조선총독부 경무국에서 소...,5326,mrc-0-003083,창씨개명령의 시행일을 미루는 것을 수락한 인물은?,윤치호
238,563,"200,000명",빌카밤바 지역은 파차쿠티 황제 때 부터 잉카 제국에 속해있던 지역이었다. 스페인 군...,48575,mrc-0-002978,망코 잉카가 쿠스코를 되찾기 위해 마련한 군사는 총 몇 명인가?,신잉카국


In [3]:
wiki_data

,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,None,나라 목록,None,None,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,None,나라 목록,None,None,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,TODO,None,백남준,None,None,2
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,TODO,None,아오조라 문고,None,None,3
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,TODO,None,아오조라 문고,None,None,4
...,...,...,...,...,...,...,...,...
60608,"오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...",위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60608
60609,"1936년 7월, 오키나와현 교육위원회 부설 향토 박물관(沖縄県教育会附設郷土博物館)...",위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60609
60610,박물관은 개관 10주년이되는 2017년에 관의 애칭 및 마스코트를 일반인에게 공모했...,위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60610
60611,단결권 및 단체교섭권 협약(Right to Organise and Collectiv...,위키피디아,None,None,단결권 및 단체교섭권 협약,None,None,60611


## 먼저 Train, Valid Data에 context가 wikidocs에 잘 있는지 확인한다.

- 확인을 해보니 wiki docs에는 정확히 있는 문단 같음.

In [4]:
wiki_texts = wiki_data['text'].values
cnt = 0
not_exist_contexts = []
for i in range(len(data)):
    text = data.iloc[i]['context']
    if text in wiki_texts:
        cnt += 1
    else:
        not_exist_contexts.append(text)
print(len(data))
print(cnt)
print(len(not_exist_contexts))

4192
4192
0


# TF-IDF를 활용해서 retrieval을 구현한다.

In [5]:
from Tfidf import SparseRetrieval
from transformers import AutoTokenizer
import json

model_checkpoint = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

SparseRetrieval(wikidocs_path, tokenizer)

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


TypeError: sequence item 0: expected str instance, tokenizers.Encoding found